The first step in wrangling our EEG results is to import our datasets. They have been separated into 3 files, `run4.csv`, `run5.csv`, and `run4_5.txt`. The reason for this is that we wrote a `keylogger` (a program that captures the timing of keyboard inputs) that writes to one file, and we separated out the runs in the `OpenBCI` program. 

To import the raw EEG data, we will be using the `Pandas` Python module to read the raw CSV file into a `DataFrame`, which is an easily accessible large-scale format object for managing datasets. 

Once we have the DataFrames for the 2 selected runs, we need to combine them, so that we can access the entire dataset at once. `Pandas` will also help with this

In [141]:
import pandas as pd # Import Pandas to project
import numpy as np                                      
import matplotlib.pyplot as plt                         
import matplotlib.patches as patches
import seaborn as sns
import scipy.signal as signal 
from scipy.io import loadmat
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from pathlib import Path
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [142]:
# Read run4.csv into Pandas DataFrame
run_3 = pd.read_csv(r'dataset/run3.csv')
run_4 = pd.read_csv(r'dataset/run4.csv')
run_5 = pd.read_csv(r'dataset/run5.csv')

eeg_data = pd.concat([run_4, run_5])

eeg_data 

,Sample Index,EXG Channel 0,EXG Channel 1,EXG Channel 2,EXG Channel 3,EXG Channel 4,EXG Channel 5,EXG Channel 6,EXG Channel 7,Accel Channel 0,...,Other.3,Other.4,Other.5,Other.6,Analog Channel 0,Analog Channel 1,Analog Channel 2,Timestamp,Other.7,Timestamp (Formatted)
0,171.00000,6179.94442,2068.16221,-9348.55006,-324.45792,-591.33775,-1583.41993,-12547.66584,-13145.91028,0.05200,...,7.00000,208.00000,30.00000,208.00000,0.00000,0.00000,0.00000,1678313733.21671,0.00000,2023-03-08 14:15:33.216
1,172.00000,6184.52653,2071.13499,-9344.77262,-283.77775,-575.13274,-1581.56473,-12546.43649,-13146.46907,0.05200,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1678313733.21674,0.00000,2023-03-08 14:15:33.216
2,173.00000,6185.86763,2072.36434,-9343.85620,-361.22654,-581.57004,-1584.02343,-12544.93893,-13149.68773,0.05200,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1678313733.21675,0.00000,2023-03-08 14:15:33.216
3,174.00000,6186.67229,2071.64908,-9342.15746,-430.98634,-593.55057,-1585.36453,-12543.15079,-13155.65564,0.05200,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1678313733.21676,0.00000,2023-03-08 14:15:33.216
4,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.05200,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1678313733.29276,0.00000,2023-03-08 14:15:33.292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86972,184.00000,-1268.39444,-3476.34446,-9734.16236,-2671.12287,-4573.34573,-6089.30809,-13128.69944,-13841.11659,0.05800,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1678315269.78043,0.00000,2023-03-08 14:41:09.780
86973,185.00000,-1268.50620,-3475.18217,-9729.64731,-3879.41347,-4632.98018,-6111.30221,-13127.22422,-13850.01258,0.05800,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1678315269.78534,0.00000,2023-03-08 14:41:09.785
86974,186.00000,-1269.06499,-3474.95866,-9731.54720,-4805.75917,-4645.83244,-6106.02720,-13128.38651,-13845.02814,0.05800,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1678315269.78930,0.00000,2023-03-08 14:41:09.789
86975,187.00000,-1265.39931,-3476.85855,-9737.09044,-3011.27172,-4561.36519,-6075.49472,-13130.28641,-13830.83479,0.05800,...,8.00000,208.00000,30.00000,128.00000,0.00000,0.00000,0.00000,1678315269.79332,0.00000,2023-03-08 14:41:09.793


Now that the raw CSV files have been read into DataFrames, as can be seen above, there are many columns (or channels) of data, not all of which are necessary for our purposes, which are to read the raw EEG data. Given that, we can actually drop some of these columns, using the `df.drop()` function from Pandas. Doing this will increase the speed of our lookup algorithm in future cells. 

One problem this dataset has is that all of the column names were inputted to the CSV file with a space in the front, so we need to go and remove the space from each column name.

We only want the raw electrode data, and therefore only want the `EXG` channels, not the `accel` channels. Rather than manually input every channel to drop, we can instead filter the columns of the `DataFrame` by whether they include the `Accel, Other, Analog,` or `Formatted`, and then use that filtered `List` as the list of columns to drop!

In [143]:
eeg_cols_raw = eeg_data.columns # Read DataFrame columns into List

DISALLOWED_PHRASES = ['Accel', 'Other', 'Analog', 'Formatted', 'Sample'] # List of keywords not allowed in columns

eeg_cols_filtered = [] # Final Column list

for col in eeg_cols_raw: # For each column
    append = False
    for phrase in DISALLOWED_PHRASES: # For each phrase in the list
        if phrase in col: # If that phrase is in the column string
            append = True 
            break
    if append:
        eeg_cols_filtered.append(col)


eeg_data = eeg_data.drop(eeg_cols_filtered, axis=1) # Drop unnecessary columns

stripped_cols = []

for col in eeg_data.columns:
    stripped_cols.append(col.lstrip()) # Get rid of beginning space

eeg_data.columns = stripped_cols # Set columns

# eeg_data['Timestamp'] -= 28800

eeg_data

,EXG Channel 0,EXG Channel 1,EXG Channel 2,EXG Channel 3,EXG Channel 4,EXG Channel 5,EXG Channel 6,EXG Channel 7,Timestamp
0,6179.94442,2068.16221,-9348.55006,-324.45792,-591.33775,-1583.41993,-12547.66584,-13145.91028,1678313733.21671
1,6184.52653,2071.13499,-9344.77262,-283.77775,-575.13274,-1581.56473,-12546.43649,-13146.46907,1678313733.21674
2,6185.86763,2072.36434,-9343.85620,-361.22654,-581.57004,-1584.02343,-12544.93893,-13149.68773,1678313733.21675
3,6186.67229,2071.64908,-9342.15746,-430.98634,-593.55057,-1585.36453,-12543.15079,-13155.65564,1678313733.21676
4,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1678313733.29276
...,...,...,...,...,...,...,...,...,...
86972,-1268.39444,-3476.34446,-9734.16236,-2671.12287,-4573.34573,-6089.30809,-13128.69944,-13841.11659,1678315269.78043
86973,-1268.50620,-3475.18217,-9729.64731,-3879.41347,-4632.98018,-6111.30221,-13127.22422,-13850.01258,1678315269.78534
86974,-1269.06499,-3474.95866,-9731.54720,-4805.75917,-4645.83244,-6106.02720,-13128.38651,-13845.02814,1678315269.78930
86975,-1265.39931,-3476.85855,-9737.09044,-3011.27172,-4561.36519,-6075.49472,-13130.28641,-13830.83479,1678315269.79332


Now that we only have the required columns for analysis in our DataFrame, the next step is to add a `KeyPressed` column, whose entries will be the key the user was pressing at the time of the reading. 

A huge problem is that because our KeyLogger and OpenBCI run at different sampling rates, and they may not have been started at exactly the same time for each run, and differences in the computers, the timestamps may not exactly align between the KeyLogger output, and the EEG data. 

There is no perfect solution for this, but we feel that one that works well enough for our purposes is to get the keypress with the closest TimeStamp to our current sample. 

The first step to this is to read the KeyLogger result into two `Lists`, where the indeces match up exactly. The `timestamps` `List` will contain the `UNIX Timestamps` for each keypress converted into a DateTime object, and the `keys` `List` will contain the key pressed at that time. 

The reason we are using a `List` and not a `Dictionary` is that `Lists` are designed to be indexed and searched quickly, whereas `Dictionaries` often take much more time, and performing logical operations on them en-masse is very inefficient.

In [144]:
from datetime import datetime

timestamps = []
keys = []

with open(r'dataset/run4_5.txt') as key_file:
    for line in key_file:
        key = line.split(',')[0].replace("'", "")
        timestamp = float(line.split(',')[1].replace('\n', ''))

        timestamps.append(timestamp)
        keys.append(key)


obj = { 'Timestamp': timestamps, 'key': keys }

key_pressed = pd.DataFrame(data=obj)

key_pressed.sort_values('Timestamp')

key_pressed
print(key_pressed['key'].unique())


['f' 'w' '/' 'g' 'a' 'm' 'e' 'Key.tab' 'Key.space' 'c' 'Key.enter' 'd' 's'
 'u' 'r' 'Key.shift' 'A' 'W' 'S' 'D' 'Button.left' 'Button.right']


Now that we have the timestamps and keys imported and readable, we need to add the `KeyPressed` column, that contains the appropriate key.

In [145]:
def key_code(key):
    if (key == "w") | (key == "a") | ('key' == "s") | (key == "d") | (key == "W") | (key == "A") | ('key' == "S") | (key == "D"):
        return 1
    elif key == 'Button.left':
        return 2
    elif key == 'Button.right':
        return 3
    else:
        return 4
   
key_pressed['key'] = key_pressed['key'].apply(key_code)
to_drop = key_pressed.loc[key_pressed['key'] == 4]

key_pressed = key_pressed.drop(to_drop.index).reset_index(drop=True)
key_pressed.head()
print(key_pressed['key'].unique())

[1 2 3]


In [146]:
def get_closest_key_press(key, df_ts):
    '''
        Steps:
            - Find closest timestamp to curkey timestamp in eeg_data rows
            - Put key pressed as the current key in iteration in eeg_data
    '''
    result_index = eeg_data['Timestamp'].sub(df_ts).abs().idxmin()
    eeg_data['KeyPressed'][result_index] = key

def back_fill_empties(keyPressed):
    print(keyPressed)
    if keyPressed == 'None':
        return 'NO_KEY'
    return keyPressed

eeg_data['KeyPressed'] = None

key_pressed.apply(lambda x: get_closest_key_press(x['key'], x['Timestamp']), axis=1)

eeg_data['KeyPressed'] = eeg_data['KeyPressed'].fillna(value='NO_BUTTON')

eeg_data.to_csv('eeg_val_to_key_press.csv')

In [147]:
eeg = eeg_data
print(eeg['KeyPressed'].unique())
eeg.head()

[2.0 'NO_BUTTON' 3.0 1.0]


,EXG Channel 0,EXG Channel 1,EXG Channel 2,EXG Channel 3,EXG Channel 4,EXG Channel 5,EXG Channel 6,EXG Channel 7,Timestamp,KeyPressed
0,6179.94442,2068.16221,-9348.55006,-324.45792,-591.33775,-1583.41993,-12547.66584,-13145.91028,1678313733.21671,2.00000
1,6184.52653,2071.13499,-9344.77262,-283.77775,-575.13274,-1581.56473,-12546.43649,-13146.46907,1678313733.21674,NO_BUTTON
2,6185.86763,2072.36434,-9343.85620,-361.22654,-581.57004,-1584.02343,-12544.93893,-13149.68773,1678313733.21675,NO_BUTTON
3,6186.67229,2071.64908,-9342.15746,-430.98634,-593.55057,-1585.36453,-12543.15079,-13155.65564,1678313733.21676,NO_BUTTON
4,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1678313733.29276,NO_BUTTON


Resampling for constant frequency

In [148]:
import mne
data_path = r'eeg_val_to_key_press.csv'

# Load the CSV data into a NumPy array
data = pd.DataFrame(eeg['Timestamp'], columns = ['Timestamp'])
data[eeg.columns[:8]] = eeg[eeg.columns[:8]]
data=np.array(data)

# Define the time values for each sample
time_values = eeg['Timestamp']

# Calculate the average time between samples
avg_delta_t = np.mean(np.diff(time_values))

# Create a MNE-Python Info object to store information about the data
n_channels = 8  # Set the number of EEG channels
ch_names = ['Fp1', 'T3', 'C3', 'C4', 'T4', 'Fz', 'Cz', 'Pz']  # Set the channel names
ch_types = ['eeg'] * n_channels  # Set the channel types to 'eeg' for all channels
info = mne.create_info(ch_names=ch_names, sfreq=1/avg_delta_t, ch_types=ch_types)

# Create a MNE-Python RawArray object to store the EEG data
data_values = data[:, 1:]
raw = mne.io.RawArray(data_values.T, info)

# Resample the data to a new sampling rate (in Hz)
new_fs = 250
raw_resampled = raw.resample(new_fs)

# Print some basic information about the EEG data
print(raw_resampled.info)

Creating RawArray with float64 data, n_channels=8, n_times=328825
    Range : 0 ... 328824 =      0.000 ...  1536.580 secs
Ready.
<Info | 7 non-empty values
 bads: []
 ch_names: Fp1, T3, C3, C4, T4, Fz, Cz, Pz
 chs: 8 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 107.0 Hz
 meas_date: unspecified
 nchan: 8
 projs: []
 sfreq: 250.0 Hz
>


In [170]:
events = pd.DataFrame(list(eeg['Timestamp'].loc[eeg['KeyPressed'] != 'NO_BUTTON']), columns = ['Timestamp'])

events['duration'] = 5

events['key'] = list(eeg['KeyPressed'].loc[eeg['KeyPressed'] != 'NO_BUTTON'])
events['Timestamp'] = events['Timestamp'].astype(int)
events['key'] = events['key'].astype(int)
#events_array = np.rec.fromrecords(events[['time', 'duration', 'key']].values.tolist(),
 #                                  names='onset,duration,event_id')

# read events with MNE function
#events = mne.read_events(events_array)
#events = mne.read_events(events_array)
#events = np.array(events)
#new_events = []
#for event in events:
 #   temp = []
 #   for e in event:
 #      temp.append(int(e))
 #   new_events.append(temp)
print(events)  
#events.shape

       Timestamp  duration  key
0     1678313733         5    2
1     1678313735         5    2
2     1678313736         5    2
3     1678313738         5    3
4     1678313739         5    3
...          ...       ...  ...
2050  1678315267         5    1
2051  1678315268         5    1
2052  1678315268         5    2
2053  1678315268         5    1
2054  1678315269         5    2

[2055 rows x 3 columns]


In [159]:
raw_resampled.times
sdt = raw_resampled.times[1] - raw_resampled.times[0]
sdt

0.004

In [171]:
mne.write_events('events.fif', events, overwrite=True)

Overwriting existing file.


C:\Users\Carly Freedman\AppData\Local\Temp\ipykernel_30684\3511839828.py:1: RuntimeWarning: This filename (c:\Users\Carly Freedman\Documents\GitHub\Winter 23\COGS 189\Final-Project-Cogs-189\events.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst or _eve.txt
  mne.write_events('events.fif', events, overwrite=True)


In [173]:
events = mne.read_events('events.fif')
event_id = {'Move': 1, 'Place Block': 2, 'Swing Sword':3}
tmin, tmax = -0.5, 1.0
baseline = (None, 0)
epochs = mne.Epochs(raw_resampled, events, event_id, tmin, tmax, baseline=baseline, event_repeated='drop')

# View the epochs
epochs

Multiple event values for single event times found. Keeping the first occurrence and dropping all others.
Not setting metadata
1104 matching events found
Setting baseline interval to [-0.5, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated


C:\Users\Carly Freedman\AppData\Local\Temp\ipykernel_30684\2296872003.py:1: RuntimeWarning: This filename (events.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events('events.fif')


Number of events,1104
Events,Move: 648Place Block: 198Swing Sword: 258
Time range,-0.500 – 1.000 sec
Baseline,-0.500 – 0.000 sec
